In [1]:
import itertools
import logging
import os 
import pickle
import shutil
import sys
from copy import deepcopy
from importlib import reload
import xmltodict

import numpy as np
import parmed as pmd
import ray
from openff.toolkit import ForceField as openffForceField
from openff.toolkit import Molecule as openffMolecule
from openff.toolkit import Topology as openffTopology
from openff.units import unit as openff_unit
from openmmforcefields.generators import (
    GAFFTemplateGenerator,
    SMIRNOFFTemplateGenerator,
)
from rdkit import Chem
from tqdm import tqdm

from openmm import *
from openmm.app import *
from openmm.unit import *

# os.environ["NUMEXPR_MAX_THREADS"] = "8"
os.environ["OE_LICENSE"] = os.environ["HOME"] + "oe_license.txt"


# ray.init(dashboard_port=8838, log_to_driver=True, logging_level="warning")

reload(logging)
logger = logging.getLogger("PL_ABFE-BRD4")
logging.basicConfig(
    format="%(asctime)s %(message)s", datefmt="%Y-%m-%d %I:%M:%S %p", level=logging.INFO
)
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)

force_fields = "force_fields"
initial_data = "initial_data"
prepared_data = "prepared_data"
# working_data = "/home/jta002/workspace/PL-ABFE/PL-ABFE-BRD4/results_4-13-25_OBC2/data/ucsd/gilsonlab/jta002/working_data"
# working_data = "/tscc/lustre/ddn/scratch/jta002/working-data"
working_data = "/data/ucsd/gilsonlab/jta002/working_data"
# working_data = "working_data"
results_dirname = "results"

data_dir_names = {
    "initial_data": initial_data,
    "prepared_data": prepared_data,
    "working_data": working_data,
    "results": results_dirname,
}

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html
LICENSE: No product keys!
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html


In [15]:
def add_CustomGB_force(xml_filename: str, system: openmm.System):
    with open(xml_filename, "r") as f:
        ff_xml = f.read()
    ff_dict = xmltodict.parse(ff_xml)
    
    gb_force = openmm.CustomGBForce()
    
    gb_force.addPerParticleParameter("q")
    gb_force.addPerParticleParameter("radius")
    gb_force.addPerParticleParameter("scale")
    gb_force.addPerParticleParameter("isntDummy")
    gb_force.addGlobalParameter("solventDielectric", float(ff_dict["SMIRNOFF"]["CustomGBSA"]["@solvent_dielectric"]))
    gb_force.addGlobalParameter("soluteDielectric", float(ff_dict["SMIRNOFF"]["CustomGBSA"]["@solute_dielectric"]))
    gb_force.addGlobalParameter("surface_area_penalty", 5.4)
    gb_force.addGlobalParameter("solvent_radius", 0.14)
    gb_force.addGlobalParameter("kappa", 0.0)
    gb_force.addGlobalParameter("PI", float(np.pi))
    gb_force.addGlobalParameter("ONE_4PI_EPS0", 138.935485)
    
    gb_force.addComputedValue("I", "select(step(r+sr2-or1), 0.5*(1/L-1/U+0.25*(r-sr2^2/r)*(1/(U^2)-1/(L^2))+0.5*log(L/U)/r), 0);"
                                  "U=r+sr2;"
                                  "C=2*(1/or1-1/L)*step(sr2-r-or1);"
                                  "L=max(or1, D);"
                                  "D=abs(r-sr2);"
                                  "sr2 = scale2*or2;"
                                  "or1 = radius1-0.009; or2 = radius2-0.009", CustomGBForce.ParticlePairNoExclusions)

    if ff_dict["SMIRNOFF"]["CustomGBSA"]["@gb_model"] == "OBC":
        gb_force.addComputedValue("B", f"1/(1/or-tanh({float(ff_dict['SMIRNOFF']['CustomGBSA']['@alpha'])}*psi-{float(ff_dict['SMIRNOFF']['CustomGBSA']['@beta'])}*psi^2+{float(ff_dict['SMIRNOFF']['CustomGBSA']['@gamma'])}*psi^3)/radius);"
                                      "psi=I*or; or=radius-0.009", CustomGBForce.SingleParticle)

    elif ff_dict["SMIRNOFF"]["CustomGBSA"]["@gb_model"] == "OBC_Logistic":
        gb_force.addComputedValue("B", "logistic_curve; "
                                  f"logistic_curve=(A+(K-A)/((1+nu*exp(b*(inflec-(psi))))^(1/nu))); "
                                  f"A=radius; K=((or)^(-1) - (radius)^(-1))^(-1); nu={float(ff_dict['SMIRNOFF']['CustomGBSA']['@nu'])}; b={float(ff_dict['SMIRNOFF']['CustomGBSA']['@b'])}; inflec={float(ff_dict['SMIRNOFF']['CustomGBSA']['@inflec'])}; "
                                  f"psi=I*or; or=radius-0.009;", CustomGBForce.SingleParticle)

    gb_force.addEnergyTerm("-0.5*ONE_4PI_EPS0*(1/soluteDielectric-1/solventDielectric)*q^2/B", CustomGBForce.SingleParticle)
    gb_force.addEnergyTerm("4*PI*surface_area_penalty*(radius+solvent_radius)^2*(radius/B)^6;", openmm.CustomGBForce.SingleParticle)
    gb_force.addEnergyTerm("-ONE_4PI_EPS0*(1/soluteDielectric-1/solventDielectric)*q1*q2/f;"
                          "f=sqrt(r^2+B1*B2*exp(-r^2/(4*B1*B2)))", CustomGBForce.ParticlePairNoExclusions);
    
    matches = []
    for smirks_dict in ff_dict["SMIRNOFF"]["CustomGBSA"]["Atom"]:
        matches.append([match[0] for match in sodium.chemical_environment_matches(smirks_dict["@smirks"])] + [match[0] + sodium.n_atoms for match in chloride.chemical_environment_matches(smirks_dict["@smirks"])])
    
    for i, atom in enumerate(openff_topology.atoms):
        for j, match_list in enumerate(matches):
            if i in match_list:
                specific_atom_data = ff_dict["SMIRNOFF"]["CustomGBSA"]["Atom"][j]
        q = atom.partial_charge.m_as(openff_unit.elementary_charge)
        radius = float(specific_atom_data["@radius"].split("*")[0])
        scale = float(specific_atom_data["@scale"])
        isnt_dummy = 1
        gb_force.addParticle((q, radius, scale, isnt_dummy))
    
    system.addForce(gb_force);

In [16]:
force_field = openffForceField(
    "openff-2.2.0.offxml",
    "ff14sb_off_impropers_0.0.4.offxml",  # Consider removing openff impropers
    # f"{force_fields}/GBSA-OBC2.offxml",
    load_plugins=True,
    allow_cosmetic_attributes=True,
)

sodium = openffMolecule.from_smiles(smiles="[Na+]")
sodium.partial_charges = [1] * openff_unit.elementary_charge
chloride = openffMolecule.from_smiles(smiles="[Cl-]")
chloride.partial_charges = [-1] * openff_unit.elementary_charge
openff_topology = openffTopology.from_molecules(molecules=[sodium, chloride])

# positions = np.array([[0,0,0], [0,0,1]]) * openff_unit.nanometer
# openff_topology.set_positions(positions)

system = force_field.create_openmm_system(
        openff_topology,
        charge_from_molecules=[sodium, chloride],
        allow_nonintegral_charges=True,
)

system_with_GB = deepcopy(system)
add_CustomGB_force(f"{force_fields}/GBSA-OBC2.offxml", system_with_GB)

In [32]:
for s in (system, system_with_GB):

    starting_r = 20
    delta_r = 0.01

    to_remove = []
    for i, force in enumerate(s.getForces()):
        if isinstance(force, NonbondedForce):
            force.NoCutoff = True

    to_remove.sort(reverse=True)
    for i in to_remove:
        s.removeForce(i)

    with open("temp.txt", "w") as f:
        f.write("")
    integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/nanosecond, 1*femtoseconds)
    simulation = Simulation(openff_topology.to_openmm(), s, integrator)
    simulation.context.setPositions(np.array([[0,0,0], [0,0,starting_r]]))
    simulation.reporters.append(StateDataReporter("temp.txt", reportInterval=1, potentialEnergy=True))
    simulation.step(1)
    with open("temp.txt", "r") as f:
        initial_energy = float(f.readlines()[1])

    with open("temp.txt", "w") as f:
        f.write("")
    integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/nanosecond, 1*femtoseconds)
    simulation = Simulation(openff_topology.to_openmm(), s, integrator)
    simulation.context.setPositions(np.array([[0,0,0], [0,0,starting_r-delta_r]]))
    simulation.reporters.append(StateDataReporter("temp.txt", reportInterval=1, potentialEnergy=True))
    simulation.step(1)
    with open("temp.txt", "r") as f:
        final_energy = float(f.readlines()[1])

    calculated_force = (final_energy - initial_energy) / delta_r
    print(initial_energy, calculated_force)

-6.946773052215576 -0.3474712371826172
-895.1612162590027 -0.00438690185546875


In [5]:
for force in system_with_GB.getForces():
    if isinstance(force, CustomGBForce):
        gb_force = force

gb_force.getParticleParameters(0)

(1.0, 0.15, 0.8, 1.0)